## Evolución del radio R(t)

Calculo el efecto de la presión p_g debido a las moléculas presentes en el radio (variable termodinámica) empleando la ecuación diferencial 2.2.1.1 de la tesis de Gabriela y lo aprendido en la evolución del radio para el caso de la presión eléctrica. Por lo pronto sólo expreso la presión y su derivada

### Importo dependencias

In [1]:
#Importo librerías

In [2]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [3]:
#Importo notebooks
from parametros import Rg, rol, rapidez, mu, sigma, var_termo
from conductividad_termica import p_g, calculamp
from funciones_math import Vol
from funciones_qcas import masa
from presion import p_B

importing Jupyter notebook from parametros.ipynb
1.673557692882144e-27
importing Jupyter notebook from conductividad_termica.ipynb
importing Jupyter notebook from funciones_math.ipynb
importing Jupyter notebook from funciones_qcas.ipynb


In [6]:
from parametros import cl, rol, p_inf

In [7]:
def derivada_radio(y,n,dydt):
    #Existen 2 opciones: (1) usar la versión simplificada de la ecuación de momento del Bunkin o (2) usar la expresión completa de Gabriela.

    # #Asigno variables termodinámicas
    # v_R = y[var_termo["v_R"]]
    # mp = y[var_termo["mp"]]
    
    # #Asigno valores de ctes
    # rho_L_i = rol

    # term1 = -(3/2)*v_R**2*(1 - v_R/(3*cl) + 2*mp/(3*cl*rho_L_i))
    # term2 = (1/rho_L_i)*(1 + v_R/cl)*(p_B(R,T_e_, q_e_, R0_) - p_inf)
    # term3 = (mpp(t)*R/rho_L_i)*(1 - v_R/cl + mp/(cl*rho_L_i))
    # term4 = (mp/rho_L_i)*(v_R - mp/(2*rho_L_i) + mp*v_R/(2*cl*rho_L_i))
    # term5 = (R/cl*rho_L_inf)*dpBdt(R, v_R,T_e_, q_e_, R0_)
    # dydt_1 = (1/(R*(1 - v_R/cl + mp/(cl*rho_L_i))     ))*(term1 + term2 + term3 + term4 + term5)

    #Opto por (1):
    #Sistema de ecuaciones de primer orden para la evolución del radio en función del tiempo.

    #Asigno variables termodinámicas
    R = y[var_termo["R"]]
    v_R = y[var_termo["v_R"]]
    
    #Asigno valores de ctes
    rho_L = rol

    #Expreso dydt_0 = dR/dt = v_R
    dydt[var_termo["R"]] = v_R

    #Expreso dydt_2 = d(v_R/dt) = d2(R)/dt2
    dydt[var_termo["v_R"]] = ((p_B(y, n)-p_inf)/rho_L - 3/2*v_R**2)/R

    return
